In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORTAÇÃO DA BASE DE SÓCIOS
socios_empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas.csv', dtype=str)
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,tempo_de_sociedade,cnpj_cpf_socio_mascarado
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,11,***923641**
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,11,***491386**
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,11,***718468**
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,9,***415907**
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,9,***022878**
...,...,...,...,...,...
1683063,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,10,***906288**
1683064,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,10,***950407**
1683065,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,10,***993174**
1683066,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,10,***288983**


In [3]:
# Remover coluna de tempo_de_sociedade, não necessária aqui
socios_empresas_rotuladas_df.drop(columns=['tempo_de_sociedade'], inplace=True)

### Junção com a base de empresas rotuladas

In [4]:
# IMPORTAÇÃO DA BASE DE EMPRESAS ROTULADAS
empresas_rotuladas_df = pd.read_csv(r'C:\DATASETS\Empresas Rotuladas\empresas_rotuladas.csv', dtype=str, usecols=['cnpj', 'inexistente'])
empresas_rotuladas_df

,cnpj,inexistente
0,00000942000170,True
1,00001102000121,True
2,00005201000181,True
3,00005201000262,True
4,00010501000159,True
...,...,...
1559938,98593668000194,False
1559939,98669997000252,False
1559940,98670235000195,False
1559941,98670656000116,False


In [5]:
# Rotula os sócios com a variável 'inexistente'
socios_empresas_rotuladas_df = socios_empresas_rotuladas_df.merge(empresas_rotuladas_df, on='cnpj', how='left')
print(socios_empresas_rotuladas_df.inexistente.value_counts())
print(socios_empresas_rotuladas_df)

True     1355428
False     328286
Name: inexistente, dtype: int64
                   cnpj cnpj_cpf_socio                      nome_socio  \
0        00000000000191    ***923641**        MARCIO HAMILTON FERREIRA   
1        00000000000191    ***491386**       NILSON MARTINIANO MOREIRA   
2        00000000000191    ***718468**           WALTER MALIENI JUNIOR   
3        00000000000191    ***415907**     CARLOS ALBERTO ARAUJO NETTO   
4        00000000000191    ***022878**        ANTONIO MAURICIO MAURANO   
...                 ...            ...                             ...   
1683709  97541975000169    26090628859        EDIVALDO ALVES DE ARAUJO   
1683710  97542225000101    01095040707  CARLOS HENRIQUE ROCHA DE SOUZA   
1683711  97543869000114    72399317491       MARIO JORGE LIMA DA SILVA   
1683712  97548003000104    03028898330    CARLOS ADRIANO ROSA DA SILVA   
1683713  97549681000183    90619544449          JOSENILDO JOSE DE LIMA   

        cnpj_cpf_socio_mascarado inexistente 

In [7]:
# Consolida os sócios
socios_empresas_rotuladas_df.inexistente = np.where(socios_empresas_rotuladas_df.inexistente=='True', 1, 0)
                                              .astype(int)
participa_inexistente_df = socios_empresas_rotuladas_df.groupby(['cnpj_cpf_socio_mascarado', 'nome_socio'])
                                              .agg({'inexistente': 'sum'}).reset_index()
participa_inexistente_df

,cnpj_cpf_socio_mascarado,nome_socio,inexistente
0,***000000**,ANA BEATRIZ VERISSIMO SCHMITT,1
1,***000001**,MARCOS VINICIUS DIAS BARBOSA,1
2,***000001**,OTAVIO AUGUSTO RODRIGUES SAMPAIO DE SOUZA,1
3,***000002**,ANDREZA DA CONCEICAO SANTOS,1
4,***000003**,ODELIDE CONCEICAO DA SILVA,1
...,...,...,...
1608864,97837181000147,DURATEX S.A.,0
1608865,97929203000107,SUNE SOLAR BV,0
1608866,98042120000156,COOPERATIVA AGRO-PECUARIA ALTO URUGUAI LTDA EM...,0
1608867,98408073000111,"VULCABRAS AZALEIA-RS,CALCADOS E ARTIGOS ESPORT...",0


In [8]:
# Há muitos sócios que participam de várias empresas inexistentes?
participa_inexistente_df[participa_inexistente_df.inexistente>1]

,cnpj_cpf_socio_mascarado,nome_socio,inexistente
1312,***000811**,CLELIA LEMOS GUSMAO,4
1445,***000897**,VALMIR FERREIRA,4
1567,***000968**,DAVID ANTONIO BALDO,2
4544,***002831**,YARALICE ANTUNES DA COSTA,4
4916,***003061**,PAULO ROBERTO DOS REIS GAMA,2
...,...,...,...
1606334,29444965000155,ADMINISTRADORA SELCO LTDA,3
1606525,30625804000147,PLATINA METAIS E CONDUTORES ELETRICOS EIRELI,4
1607120,33822065000171,ADPART-ADMINISTRACAO E PARTICIPACAO LTDA,2
1607691,41110248000192,GRANDE RIO-PARTICIPACAO E ADMINISTRACAO LTDA,2


559 sócios participam de mais de uma empresa inexistente.

In [9]:
# Renomeia coluna
participa_inexistente_df.rename(columns={'inexistente': 'empresas_inexistentes_socio_participa'}, inplace=True)

### Faz o join com a base de socios de empresas rotuladas:

In [10]:
socios_empresas_rotuladas_df = socios_empresas_rotuladas_df.merge(participa_inexistente_df, on=['cnpj_cpf_socio_mascarado', 'nome_socio'], how='left')
socios_empresas_rotuladas_df

,cnpj,cnpj_cpf_socio,nome_socio,cnpj_cpf_socio_mascarado,inexistente,empresas_inexistentes_socio_participa
0,00000000000191,***923641**,MARCIO HAMILTON FERREIRA,***923641**,0,0
1,00000000000191,***491386**,NILSON MARTINIANO MOREIRA,***491386**,0,0
2,00000000000191,***718468**,WALTER MALIENI JUNIOR,***718468**,0,0
3,00000000000191,***415907**,CARLOS ALBERTO ARAUJO NETTO,***415907**,0,0
4,00000000000191,***022878**,ANTONIO MAURICIO MAURANO,***022878**,0,0
...,...,...,...,...,...,...
1683709,97541975000169,26090628859,EDIVALDO ALVES DE ARAUJO,***906288**,0,0
1683710,97542225000101,01095040707,CARLOS HENRIQUE ROCHA DE SOUZA,***950407**,0,0
1683711,97543869000114,72399317491,MARIO JORGE LIMA DA SILVA,***993174**,0,0
1683712,97548003000104,03028898330,CARLOS ADRIANO ROSA DA SILVA,***288983**,0,0


In [33]:
# Quantos sócios por número de empresas inexistentes que participa?
socios_empresas_rotuladas_df.empresas_inexistentes_socio_participa.value_counts()

1     1355834
0      325873
2         614
5         278
3         255
4         211
7         154
6         115
8          96
13         65
11         55
9          45
12         36
15         30
10         30
23         23
Name: empresas_inexistentes_socio_participa, dtype: int64

In [34]:
# Curiosidade:
socios_empresas_rotuladas_df[socios_empresas_rotuladas_df.empresas_inexistentes_socio_participa == 23]

,cnpj,cnpj_cpf_socio,nome_socio,cnpj_cpf_socio_mascarado,inexistente,empresas_inexistentes_socio_participa
32859,02913970000103,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32860,02913970000294,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32861,02913970000375,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32862,02913970000456,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32863,02913970000537,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32864,02913970000618,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32865,02913970000707,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32866,02913970000880,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32867,02913970000960,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23
32868,02913970001002,***165648**,ALBERTO ARMANDO FORTE,***165648**,1,23


In [35]:
# Não precisamos mais da coluna 'inexistente':
socios_empresas_rotuladas_df = socios_empresas_rotuladas_df.drop(columns=['inexistente'])

In [36]:
# Salvando resultado:
socios_empresas_rotuladas_df.to_csv(r'C:\DATASETS\Empresas Rotuladas\socios_empresas_rotuladas-participa_empresa_inexistente.csv', index = False)